In [1]:
import os

In [2]:
%pwd

'e:\\Karan_Bais\\python\\Ml\\Projects\\Text_summary\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Karan_Bais\\python\\Ml\\Projects\\Text_summary'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from  Text_summary.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from Text_summary.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(path_to_yaml=config_filepath)
        self.params = read_yaml(path_to_yaml=params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(

            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        
        return data_transformation_config

In [8]:
import os
from Text_summary.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

e:\Anaconda\envs\text-sum\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-08 11:17:51,857]: INFO: config: PyTorch version 2.3.0 available.]


In [9]:
class DataTransformation:
    def __init__(self, data_transformation_config: DataTransformationConfig):
        self.config = data_transformation_config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name, use_fast = False)

    def convert_text_to_feature(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length =  1024, truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation=True)
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        logger.info('Loading dataset...')
        dataset = load_from_disk(self.config.data_path)
        logger.info('Converting dataset...')
        dataset = dataset.map(self.convert_text_to_feature, batched=True)
        logger.info('Saving dataset...')
        dataset.save_to_disk(os.path.join(self.config.root_dir, 'samsum'))
        logger.info('Done!')

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-05-08 11:18:02,053]: INFO: common: yaml file: config\config.yaml loaded]
[2024-05-08 11:18:02,062]: INFO: common: yaml file: params.yaml loaded]
[2024-05-08 11:18:02,063]: INFO: common: Created directory: artifacts]
[2024-05-08 11:18:02,064]: INFO: common: Created directory: artifacts/data_transformation]
[2024-05-08 11:18:03,439]: INFO: 1037995981: Loading dataset...]
[2024-05-08 11:18:03,565]: INFO: 1037995981: Converting dataset...]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]e:\Anaconda\envs\text-sum\Lib\site-packages\transformers\tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 1512.40 examples/s]

[2024-05-08 11:18:15,017]: INFO: 1037995981: Saving dataset...]



Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 85230.17 examples/s]

[2024-05-08 11:18:15,101]: INFO: 1037995981: Done!]
